In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['iwildfastaibaseline', 'iwildcam-2019-fgvc6']


In [2]:
! ls

__notebook__.ipynb  __output__.json


In [3]:
import warnings
warnings.filterwarnings("ignore")
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from fastai.vision import *

In [5]:
PATH = Path("../input/iwildcam-2019-fgvc6")

In [6]:
df = pd.read_csv(PATH/"train.csv")

In [7]:
len(df)

196299

In [8]:
df_animals = df[df.category_id != 0] #removing null class from dataset

In [9]:
classes = df_animals["category_id"].unique()

In [10]:
classes_map = {classes[i]:i for i in range(len(classes))}

In [11]:
classes_map

{19: 0,
 3: 1,
 8: 2,
 4: 3,
 13: 4,
 1: 5,
 11: 6,
 16: 7,
 17: 8,
 14: 9,
 18: 10,
 10: 11,
 22: 12}

In [12]:
new_labels = [ classes_map[i] for i in df_animals["category_id"]]

In [13]:
df_animals["new_class"] = new_labels

In [14]:
train_df = df_animals[["file_name","new_class"]]

In [15]:
train_df.columns = ['name', 'label']

In [16]:
train_df.head()

,name,label
0,5998cfa4-23d2-11e8-a6a3-ec086b02610b.jpg,0
1,588a679f-23d2-11e8-a6a3-ec086b02610b.jpg,0
5,593d68d7-23d2-11e8-a6a3-ec086b02610b.jpg,1
6,59fae563-23d2-11e8-a6a3-ec086b02610b.jpg,2
12,595f7b9c-23d2-11e8-a6a3-ec086b02610b.jpg,3


In [17]:
data = ImageDataBunch.from_df(PATH, train_df, "train_images", size=448, bs=64)

In [18]:
from fastai.metrics import error_rate, accuracy
from fastai.vision import models 
from fastai.vision import learner

In [19]:
learn = learner.create_cnn(data, models.resnet34, metrics=[error_rate,accuracy], model_dir='/tmp/models').to_fp16() 

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /tmp/.torch/models/resnet34-333f7ec4.pth
87306240it [00:00, 88759448.95it/s]


In [20]:
# learn.lr_find()

In [21]:
# learn.recorder.plot()

In [22]:
learn.fit_one_cycle(5,1e-2)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.828236,0.871417,0.288248,0.711752,30:28
1,0.742162,0.891956,0.301743,0.698257,29:37


# Get Preds on train set
Get predicitons from train set to usse with ensemble method

In [23]:
# from fastai.vision import ImageImageList
# test_data = ImageImageList.from_df(df, PATH/"train_images", cols="file_name")


In [24]:
# learn.data.add_test(test_data)
# preds, y = learn.get_preds(ds_type=DatasetType.Test)
# np.save("animal_preds_train_res50-1", preds)

In [25]:
# interp = ClassificationInterpretation.from_learner(learn)

In [26]:
# interp.plot_confusion_matrix()

## Interperation

In [27]:
# class_map_reversed = {v:k for k,v in classes_map.items() }

In [28]:
# animals={
# 0:"nothing",
# 1: "deer",
# 2: "moose",
# 3: "squirrel",
# 4: "rodent",
# 5: "small_mammal",
# 6: "elk",
# 7: "pronghorn_antelope",
# 8: "rabbit",
# 9: "bighorn_sheep",
# 10: "fox", 
# 11: "coyote", 
# 12: "black_bear", 
# 13: "raccoon", 
# 14: "skunk", 
# 15: "wolf", 
# 16: "bobcat", 
# 17: "cat", 
# 18: "dog", 
# 19: "opossum", 
# 20: "bison", 
# 21: "mountain_goat", 
# 22: "mountain_lion"}

In [29]:
# confidence = [preds[i][j] for i,j in enumerate(final_preds)]


In [30]:
# threshold = 0.4

In [31]:
# final_preds = []
# for i,pred in enumerate(final_preds):
#     if preds[i][pred] < threshold:
#         final_preds.append(0)
#     else:
#         final_preds.append(class_map_reversed[pred])

In [32]:
# final_preds[:10]


In [33]:
# class_animal = [animals[class_map_reversed[i]] for i in final_preds.numpy()]
# 

In [34]:
# data.valid_dl.x


In [35]:
# df_dict = {}
# df_dict["correct"] = correct_preds
# df_dict["animal"] = class_animal
# df_dict["confidence"] = confidence

In [36]:
# df = pd.DataFrame.from_dict(df_dict)

In [37]:
# df.tail(n=15)

In [38]:
# data.valid_dl.x[12967]

## Submission

In [39]:
df_test = pd.read_csv(PATH/"test.csv")

In [40]:
from fastai.vision import ImageImageList
test_data = ImageImageList.from_df(df_test, PATH/'test_images', cols="file_name")
learn.data.add_test(test_data)
preds, y = learn.get_preds(ds_type=DatasetType.Test)

In [41]:
np.save("test-animal-preds-res50", preds)

In [42]:
# preds = np.load("../input/iwildfastaibaseline/preds-baseline-res50.npy")

In [43]:
# max_preds = np.argmax(preds, axis=1)

In [44]:
# confidence = [preds[i][j] for i,j in enumerate(max_preds)]

In [45]:
# class_map_reversed = {v:k for k,v in classes_map.items() }

In [46]:
# class_map_reversed

In [47]:
# threshold = 0.60
# final_preds = []
# for i,pred in enumerate(max_preds):
#     if preds[i][pred] < threshold:
#         final_preds.append(0)
#     else:
#         final_preds.append(class_map_reversed[pred])

In [48]:
# submdf = pd.read_csv("../input/iwildcam-2019-fgvc6/test.csv")

In [49]:
# df_test["Predicted"] =  final_preds


In [50]:
# df_subm = df_test[["id","Predicted" ]]
# df_subm.columns = [["Id","Predicted" ]]

In [51]:
# df_subm.head()

In [52]:
# df_subm.to_csv("aniaml-predsres50.csv", index=False)